In [1]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
    
@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [4]:
#np_random每次执行都是一样的结果。
np_random()
np_random()

array([[ 0.71765082, -0.90910153, -0.20729723],
       [ 0.64879166,  0.46155895,  0.00371346],
       [-0.06288097,  0.06418557,  1.18332953]])
array([[ 0.71765082, -0.90910153, -0.20729723],
       [ 0.64879166,  0.46155895,  0.00371346],
       [-0.06288097,  0.06418557,  1.18332953]])


In [6]:
#tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[-2.09935164 -0.540380657 -0.289289206]
 [0.133008286 -1.52922654 -2.1896472]
 [0.423097044 1.04938543 -1.28914022]]
[[-1.10056579 -1.24592519 -0.416950792]
 [-0.554157794 1.51002252 -0.177978218]
 [0.0130170025 0.232789859 0.612563848]]


In [8]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [9]:
outer_var()

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [10]:
outer_var()

5


<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [11]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#执行将报错


In [12]:
#inner_var()
inner_var()

ValueError: in user code:

    <ipython-input-11-c9e8effade57>:3 inner_var  *
        x = tf.Variable(1.0,dtype = tf.float32)
    /root/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/variables.py:268 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /root/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/variables.py:250 _variable_v2_call
        return previous_getter(
    /root/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    /root/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py:764 invalid_creator_scope
        raise ValueError(

    ValueError: tf.function-decorated function tried to create variables on non-first call.


In [13]:
#被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。

In [14]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [15]:
tensor_list = []

@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
